<a href="https://colab.research.google.com/github/douglas-regatieri/IA_Curso_Data_Science_Copa_do_Mundo_2022/blob/main/Projeto_IA_Jogos_Copa_do_Mundo_Qatar_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

selecoes = pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name='selecoes', index_col=0)
jogos = pd.read_excel('/content/DadosCopaDoMundoQatar2022.xlsx', sheet_name='jogos')

In [ ]:
selecoes.head()

In [ ]:
jogos.head()

In [ ]:
np.random.poisson(lam = 1, size = 1)

array([0])

In [ ]:
fifa = selecoes['PontosRankingFIFA'] #Transformação linear de escala numérica
a, b = min(fifa), max(fifa)
fa, fb = 0.15, 1

b1 = (fb - fa)/(b-a)
b0 = fb - b*b1
forca = b0 + b1*fifa

forca

Seleção
Catar             0.243596
Equador           0.285618
Senegal           0.516742
Holanda           0.696292
Inglaterra        0.807079
Irã               0.467079
Estados Unidos    0.612247
País de Gales     0.511011
Argentina         0.872022
Arábia Saudita    0.232135
México            0.640899
Polônia           0.442247
França            0.860562
Dinamarca         0.669551
Tunísia           0.369663
Austrália         0.323820
Espanha           0.768876
Alemanha          0.658090
Japão             0.459438
Costa Rica        0.354382
Bélgica           0.969438
Canadá            0.304719
Marrocos          0.465169
Croácia           0.606517
Brasil            1.000000
Sérvia            0.449888
Suíça             0.585506
Camarões          0.325730
Portugal          0.696292
Gana              0.150000
Uruguai           0.623708
Coreia do Sul     0.404045
Name: PontosRankingFIFA, dtype: float64

In [ ]:
def MediasPoisson(selecao1, selecao2):
  forca1 = forca[selecao1]
  forca2 = forca[selecao2]
  mgols = 2.75
  l1 = mgols*forca1/(forca1 + forca2)
  l2 = mgols - l1

  return [l1, l2]

def Resultado(gols1, gols2):
    if gols1 > gols2:
      resultado = 'V'
    elif gols2 > gols1:
      resultado = 'D'
    else:
      resultado = 'E'
    return resultado

def Pontos(gols1, gols2):
    rst = Resultado(gols1, gols2)
    if rst == 'V':
      pontos1, pontos2 = 3, 0
    elif rst == 'D':
      pontos1, pontos2 = 0, 3
    else:
      pontos1, pontos2 = 1, 1
    return [pontos1, pontos2, rst]

In [ ]:
MediasPoisson('Brasil', 'Suíça')

[1.7344624760824887, 1.0155375239175113]

In [ ]:
def Jogo(selecao1, selecao2):
  l1, l2 = MediasPoisson(selecao1, selecao2)
  gols1 = int(np.random.poisson(lam = l1, size = 1))
  gols2 = int(np.random.poisson(lam = l2, size = 1))
  saldo1 = gols1 - gols2
  saldo2 = -saldo1
  pontos1, pontos2, resultado = Pontos(gols1,gols2)
  placar = '{}X{}'.format(gols1, gols2)
  return [gols1, gols2, saldo1, saldo2, pontos1, pontos2, resultado, placar]

In [ ]:
Jogo('Brasil','Suíça')

[3, 0, 3, -3, 3, 0, 'V', '3X0']

In [ ]:
#Probability mass function
def Distribuicao(media):
  probs = []
  for i in range(7):
    probs.append(poisson.pmf(i, media)) 
  probs.append(1 - sum(probs))
  return pd.Series(probs, index = ['0', '1','2','3','4','5','6','7+'])

def ProbabilidadesPartidas(selecao1, selecao2):
  l1, l2 = MediasPoisson(selecao1,selecao2)
  d1, d2 = Distribuicao(l1), Distribuicao(l2)
  matriz = np.outer(d1, d2)
  vitoria = np.tril(matriz).sum()-np.trace(matriz) #Soma o triangulo inferior - tril = triangulo lower
  derrota = np.triu(matriz).sum()-np.trace(matriz) #Soma o triangulo superior - triu = triangulo upper
  empate = 1 - (vitoria + derrota)

  probs = np.around([vitoria, empate, derrota], 3) #Arredondar em 3 casas decimais
  probsp = [f'{100*i:.1f}%' for i in probs] #For para exibir em porcentagem e 1 casa decimal

  nomes = ['0', '1','2','3','4','5','6','7+']
  matriz = pd.DataFrame(matriz, columns=nomes, index=nomes)
  matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
  matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns])

  output = {'selecao1': selecao1, 'selecao2': selecao2,
            'f1': forca[selecao1], 'f2': forca[selecao2],
            'media1': l1, 'media2': l2,
            'probabilidades': probsp, 'matriz': matriz}
  return output

In [ ]:
ProbabilidadesPartidas('Brasil','Suíça')

{'selecao1': 'Brasil',
 'selecao2': 'Suíça',
 'f1': 1.0,
 'f2': 0.5855056179775282,
 'media1': 1.7344624760824887,
 'media2': 1.0155375239175113,
 'probabilidades': ['54.2%', '23.7%', '22.1%'],
 'matriz':               Suíça                                                    \
                   0         1         2         3         4         5   
 Brasil 0   0.063928  0.064921  0.032965  0.011159  0.002833  0.000575   
        1   0.110880  0.112603  0.057176  0.019355  0.004914  0.000998   
        2   0.096159  0.097653  0.049585  0.016785  0.004262  0.000866   
        3   0.055595  0.056459  0.028668  0.009704  0.002464  0.000500   
        4   0.024107  0.024481  0.012431  0.004208  0.001068  0.000217   
        5   0.008362  0.008492  0.004312  0.001460  0.000371  0.000075   
        6   0.002417  0.002455  0.001247  0.000422  0.000107  0.000022   
        7+  0.000759  0.000771  0.000391  0.000132  0.000034  0.000007   
 
                                    
                 

In [ ]:
jogos['Vitória'] = None
jogos['Empate'] = None
jogos['Derrota'] = None

for i in range(jogos.shape[0]):
  selecao1 = jogos['seleção1'][i]
  selecao2 = jogos['seleção2'][i]
  v, e, d = ProbabilidadesPartidas(selecao1, selecao2,)['probabilidades']
  jogos.at[i, 'Vitória'] = v
  jogos.at[i, 'Empate'] = e
  jogos.at[i, 'Derrota'] = d

jogos.to_excel('outputEstimativasJogosCopa.xlsx')
jogos

,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%
